In [ ]:
# 常用api测试

In [1]:
import tensorflow as tf 
from tensorflow.python.ops import nn 


def listwise_softmax_cross_entropy3(session_id, labels, logits, weights=None, sample_mask=None, logits_weight=1.0):
    _EPSILON = 1e-10
    if sample_mask is not None:
        session_id = tf.reshape(tf.boolean_mask(session_id, sample_mask), [-1, 1])
        labels = tf.reshape(tf.boolean_mask(labels, sample_mask), [-1, 1])
        logits = tf.reshape(tf.boolean_mask(logits, sample_mask), [-1, 1])
        if weights is not None:
            weights = tf.reshape(tf.boolean_mask(weights, sample_mask), [-1, 1])

    session_mask = tf.equal(session_id, tf.transpose(session_id))
    session_mask_f = tf.cast(session_mask, tf.float32)

    logits = tf.reshape(logits, [-1, 1])
    logits_bb = logits - tf.zeros_like(tf.transpose(logits))
    logits_final = tf.where(session_mask, logits_bb, tf.log(_EPSILON) * tf.ones_like(logits_bb))
    
    labels = tf.reshape(labels, [-1, 1])
    labels_bb = labels - tf.zeros_like(tf.transpose(labels))
    labels_final = tf.where(session_mask, labels_bb, tf.zeros_like(labels_bb))

    if weights is not None:
        weights = tf.reshape(weights, [-1, 1])
        weights_bb = weights - tf.zeros_like(tf.transpose(weights))
        weights_final = tf.where(session_mask, weights_bb, tf.zeros_like(weights_bb))
    else:
        weights_final = tf.ones_like(labels_final)

    label_sum = tf.reduce_sum(input_tensor=labels_final, axis=0, keep_dims=True)
    nonzero_mask = tf.greater(tf.reshape(label_sum, 
[-1]
), 0.0)
    padded_labels = tf.where(nonzero_mask, labels_final, _EPSILON * tf.ones_like(labels_final)) * session_mask_f
    padded_label_sum = tf.reduce_sum(input_tensor=padded_labels, axis=0, keep_dims=True)
    normalized_labels = padded_labels / padded_label_sum

    exps = tf.exp(logits_weight * logits_final) * session_mask_f
    softmax = tf.div_no_nan(exps, tf.reduce_sum(exps, axis=0))
    losses = -tf.reduce_sum(normalized_labels * tf.log(softmax + _EPSILON) * weights_final * session_mask_f, axis=0)

    per_row_weights = tf.reduce_sum(session_mask_f, axis=1)
    session_cnt = tf.reduce_sum(1.0 / per_row_weights)
    listwise_loss = tf.reduce_sum(losses / per_row_weights) / session_cnt

    return listwise_loss
